# Modeling

Based on our data analysis in the exploration.ipynb notebook, we will now build a model to predict the target label based on input features. We will use a neural network model.

We'll also build a preprocessing pipeline to transform the data before feeding it into the model.

In [1]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import pandas as pd

TRAIN_CSV = 'data/train.csv'
TEST_CSV = 'data/test.csv'

drop_cols = ["Vicuna", "Wallaby", "Turkey", "Tick"]

def proc_pipeline(filename: str, cols=None):
    df = pd.read_csv(filename)
    df = df.drop(columns=drop_cols)
    df = df.astype({'Tiglon': 'string'})
    df.loc[df['Tiglon'].isnull(), 'Tiglon'] = 'True'

    numerical_cols = df.select_dtypes(include='number').columns.drop('target')
    categorical_cols = df.select_dtypes(exclude='number').columns

    # One-hot encoding
    df_encoded = pd.get_dummies(df, columns=categorical_cols)

    # merge numerical and ohe columns
    df = pd.concat([df[numerical_cols], df_encoded], axis=1)

    # drop columns that are not in training set
    if cols is not None:
        extra_cols = set(df.columns) - set(cols)
        df = df.drop(columns=extra_cols)

    weight_0 = 1 - (df['target']==0).sum() / df.shape[0]
    weight_1 = 1 - weight_0
    # calculate class weights
    class_weights = {
        0: weight_0,
        1: weight_1
    }

    x = tf.convert_to_tensor(df.drop(columns=['target']).values, dtype=tf.float32)
    y = tf.convert_to_tensor(df['target'].values, dtype=tf.int32)
    return x, y, class_weights, df.columns

X_train, y_train, class_weights, train_cols = proc_pipeline(TRAIN_CSV)

2023-11-13 23:41:39.324608: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-13 23:41:39.378051: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 23:41:39.378091: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 23:41:39.378122: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 23:41:39.387844: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-13 23:41:39.389265: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

### Training

In [2]:
# create and adapt a normalization layer
normalizer = tf.keras.layers.Normalization()
normalizer.adapt(X_train)

In [3]:
# define model
model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

model.fit(X_train, y_train, epochs=10, batch_size=64, class_weight=class_weights)

Epoch 1/10
110/110 [==============================] - 2s 10ms/step - loss: 3.0452 - accuracy: 0.8399
Epoch 2/10
110/110 [==============================] - 1s 10ms/step - loss: 0.1134 - accuracy: 0.8943
Epoch 3/10
110/110 [==============================] - 1s 10ms/step - loss: 0.0976 - accuracy: 0.8963
Epoch 4/10
110/110 [==============================] - 1s 11ms/step - loss: 0.0932 - accuracy: 0.8999
Epoch 5/10
110/110 [==============================] - 1s 10ms/step - loss: 0.0903 - accuracy: 0.9009
Epoch 6/10
110/110 [==============================] - 1s 10ms/step - loss: 0.0890 - accuracy: 0.9028
Epoch 7/10
110/110 [==============================] - 1s 10ms/step - loss: 0.0861 - accuracy: 0.9039
Epoch 8/10
110/110 [==============================] - 1s 10ms/step - loss: 0.0863 - accuracy: 0.9009
Epoch 9/10
110/110 [==============================] - 1s 11ms/step - loss: 0.0844 - accuracy: 0.9048
Epoch 10/10
110/110 [==============================] - 1s 10ms/step - loss: 0.0827 - accura

### Evaluation

In [4]:
X_test, y_test, _, _ = proc_pipeline(TEST_CSV, train_cols)
model.evaluate(X_test, y_test)

33/33 [==============================] - 0s 4ms/step - loss: 0.5374 - accuracy: 0.7498


[0.5374284386634827, 0.7497575283050537]

We were able to achieve an accuracy of 74.98% on the test set using a 4-layer neural network.